<a href="https://colab.research.google.com/github/rafiqhasan/AI_DL_ML_Repo/blob/master/All%2520TFX/Vertex%2520AI/Vertex_Pipe_%7C_CustomTraining_KFP_%2B_TFT_%2B_Metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kfp
!pip install google-cloud-pipeline-components
!pip install -q -U tensorflow_transform

     |████████████████████████████████| 233 kB 5.2 MB/s 
     |████████████████████████████████| 127 kB 67.7 MB/s 
     |████████████████████████████████| 636 kB 56.4 MB/s 
     |████████████████████████████████| 105 kB 78.3 MB/s 
     |████████████████████████████████| 1.7 MB 69.3 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 7.9 MB/s 
     |████████████████████████████████| 93 kB 1.7 MB/s 
     |████████████████████████████████| 75 kB 4.9 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
  Created wheel for kfp: filename=kfp-1.7.1-py3-none-any.whl size=321909 sha256=c33a38a6104a04e2285404aa4f5d6ff16e0b74fa296ab1f4dc2497d490167df5
  Stored in directory: /root/.cache/pip/wheels/9

     |████████████████████████████████| 87 kB 3.5 MB/s 
     |████████████████████████████████| 1.3 MB 50.8 MB/s 
     |████████████████████████████████| 42 kB 1.5 MB/s 
     |████████████████████████████████| 105 kB 61.4 MB/s 
     |████████████████████████████████| 104 kB 77.5 MB/s 
     |████████████████████████████████| 104 kB 82.8 MB/s 
     |████████████████████████████████| 103 kB 76.9 MB/s 
     |████████████████████████████████| 103 kB 76.4 MB/s 
     |████████████████████████████████| 103 kB 75.7 MB/s 
     |████████████████████████████████| 103 kB 73.9 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 
     |████████████████████████████████| 97 kB 5.6 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 96 kB 6.0 MB/s 
     |████████████████████████████████| 92 kB 10.7 MB/s 
     |█████████████████████████

     |████████████████████████████████| 406 kB 5.4 MB/s 
     |████████████████████████████████| 9.7 MB 37.9 MB/s 
     |████████████████████████████████| 189 kB 52.3 MB/s 
     |████████████████████████████████| 19.0 MB 1.3 MB/s 
     |████████████████████████████████| 17.7 MB 74 kB/s 
     |████████████████████████████████| 454.4 MB 9.4 kB/s 
     |████████████████████████████████| 151 kB 57.6 MB/s 
     |████████████████████████████████| 62 kB 883 kB/s 
     |████████████████████████████████| 829 kB 41.2 MB/s 
     |████████████████████████████████| 2.3 MB 45.8 MB/s 
     |████████████████████████████████| 183 kB 51.4 MB/s 
     |████████████████████████████████| 144 kB 63.2 MB/s 
     |████████████████████████████████| 169 kB 65.0 MB/s 
     |████████████████████████████████| 267 kB 53.0 MB/s 
     |████████████████████████████████| 435 kB 64.2 MB/s 
     |████████████████████████████████| 83 kB 2.2 MB/s 
     |████████████████████████████████| 255 kB 51.1 MB/s 
     |█████████████

In [44]:
!rm -rf /content/task.py

In [45]:
%%writefile -a task.py

# Owner - Hasan Rafiq
# Load the TensorBoard notebook extension

import pandas as pd
# import seaborn as sns
import apache_beam as beam
import matplotlib.pyplot as plt
import numpy as np
import folium
import tempfile
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

from tfx_bsl.public import tfxio
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

from folium import plugins

import tensorflow as tf
import tensorflow.keras as keras

# https://www.tensorflow.org/tfx/tutorials/transform/census
def tf_preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    pickup_latitude = inputs['pickup_latitude']
    pickup_longitude = inputs['pickup_longitude']

    #Calculate unique values of attribute
    tft.vocabulary(inputs['trip_start_day'], vocab_filename='trip_start_day')

    pickup_latitude_scaled = tft.scale_to_0_1(pickup_latitude)
    pickup_longitude_scaled = tft.scale_to_0_1(pickup_longitude)
    distance = ((inputs['pickup_latitude'] - inputs['dropoff_latitude'])**2 +  (inputs['pickup_longitude'] - inputs['dropoff_longitude'])**2)**0.5

    return {
        'pickup_latitude_scaled': pickup_latitude_scaled,
        'pickup_longitude_scaled': pickup_longitude_scaled,
        'distance': distance,
        # 'dummy1': dummy1
    }

def beam_run_transform_data(train_data_file, transform_fn_output_dir):
  """Transform the data and write out as a TFRecord of Example protos.

  Read in the data using the CSV reader, and transform it using a
  preprocessing pipeline that scales numeric data and converts categorical data
  from strings to int64 values indices, by creating a vocabulary for each
  category.

  Args:
    train_data_file: File containing training data
    transform_fn_output_dir: Directory to write function
  """
 
  SCHEMA = tft.tf_metadata.dataset_metadata.DatasetMetadata(
    tft.tf_metadata.schema_utils.schema_from_feature_spec(RAW_DATA_FEATURE_SPEC)).schema

  # The "with" block will create a pipeline, and run that pipeline at the exit
  # of the block.
  with beam.Pipeline() as pipeline:
    with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
      # Create a TFXIO to read the census data with the schema. To do this we
      # need to list all columns in order since the schema doesn't specify the
      # order of columns in the csv.
      # We first read CSV files and use BeamRecordCsvTFXIO whose .BeamSource()
      # accepts a PCollection[bytes] because we need to patch the records first
      # (see "FixCommasTrainData" below). Otherwise, tfxio.CsvTFXIO can be used
      # to both read the CSV files and parse them to TFT inputs:
      # csv_tfxio = tfxio.CsvTFXIO(...)
      # raw_data = (pipeline | 'ToRecordBatches' >> csv_tfxio.BeamSource())
      csv_tfxio = tfxio.BeamRecordCsvTFXIO(
          physical_format='text',
          column_names=CSV_COLUMNS,
          schema=SCHEMA)

      # Read in raw data and convert using CSV TFXIO.  Note that we apply
      # some Beam transformations here, which will not be encoded in the TF
      # graph since we don't do the from within tf.Transform's methods
      # (AnalyzeDataset, TransformDataset etc.).  These transformations are just
      # to get data into a format that the CSV TFXIO can read, in particular
      # removing spaces after commas.
      raw_data = (
          pipeline
          | 'ReadTrainData' >> beam.io.ReadFromText(
              train_data_file, coder=beam.coders.BytesCoder())
          | 'FixCommasTrainData' >> beam.Map(
              lambda line: line.replace(b', ', b','))
          | 'DecodeTrainData' >> csv_tfxio.BeamSource())

      # Combine data and schema into a dataset tuple.  Note that we already used
      # the schema to read the CSV data, but we also need it to interpret
      # raw_data.
      raw_dataset = (raw_data, csv_tfxio.TensorAdapterConfig())

      # The TFXIO output format is chosen for improved performance.
      transform_fn = (
          raw_dataset | tft_beam.AnalyzeDataset(tf_preprocessing_fn))

      # Will write a SavedModel and metadata to transform_fn_output_dir, which can then
      # be read by the tft.TFTransformOutput class.
      _ = (
          transform_fn
          | 'WriteTransformFn' >> tft_beam.WriteTransformFn(transform_fn_output_dir))
  
###############################
##Feature engineering functions
def feature_engg_features(features):
  #Add TFT transformations
  raw_features = {}
  for key, val in features.items():
    if key not in RAW_DATA_FEATURE_SPEC:
      continue
    if isinstance(RAW_DATA_FEATURE_SPEC[key], tf.io.VarLenFeature):
      raw_features[key] = tf.RaggedTensor.from_tensor(
          tf.expand_dims(val, -1)).to_sparse()
      continue
    raw_features[key] = val
  tft_new_features = tft_layer(raw_features)

  features['pickup_longitude_scaled'] = tft_new_features['pickup_longitude_scaled'] 
  features['pickup_latitude_scaled'] = tft_new_features['pickup_latitude_scaled']
  features['distance'] = tft_new_features['distance']
  # features['dummy1'] = tft_new_features['dummy1']
  
  #Add new features( Non-TFT transformation ) -> Just for study purposes
  # features['distance'] = ((features['pickup_latitude'] - features['dropoff_latitude'])**2 +  (features['pickup_longitude'] - features['dropoff_longitude'])**2)**0.5

  return(features)

#To be called from TF
def feature_engg(features, label):
  #Add new features
  features = feature_engg_features(features)

  return(features, label)

###############################
###Data Input pipeline function

def make_input_fn(filename, mode, vnum_epochs = None, batch_size = 512):
    def _input_fn(v_test=False):     
        # Create list of files that match pattern
        file_list = tf.io.gfile.glob(filename)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = vnum_epochs # indefinitely
        else:
            num_epochs = 1 # end-of-input after this        
        
        # Create dataset from file list
        dataset = tf.compat.v1.data.experimental.make_csv_dataset(file_list,
                                                   batch_size=batch_size,
                                                   column_names=CSV_COLUMNS,
                                                   column_defaults=DEFAULTS,
                                                   label_name=LABEL_COLUMN,
                                                   num_epochs = num_epochs,
                                                   num_parallel_reads=30)
        
        dataset = dataset.prefetch(buffer_size = batch_size)

        #Feature engineering
        dataset = dataset.map(feature_engg)

        if mode == tf.estimator.ModeKeys.TRAIN:
            num_epochs = vnum_epochs # indefinitely
            dataset = dataset.shuffle(buffer_size = batch_size)
        else:
            num_epochs = 1 # end-of-input after this

        dataset = dataset.repeat(num_epochs)       
        
        #Begins - Uncomment for testing only -----------------------------------------------------<
        if v_test == True:
          print(next(dataset.__iter__()))
          
        #End - Uncomment for testing only -----------------------------------------------------<
        return dataset
    return _input_fn

# Define feature columns(Including feature engineered ones )
# These are the features which come from the TF Data pipeline
def create_feature_cols():
    #Keras format features
    k_pickup_longitude_scaled = tf.keras.Input(name='pickup_longitude_scaled', shape=(1,), dtype=tf.float32, sparse=False) #-> Sparse because VarLenFeature
    k_pickup_latitude_scaled = tf.keras.Input(name='pickup_latitude_scaled', shape=(1,), dtype=tf.float32, sparse=False) #-> Sparse because VarLenFeature
    k_month = tf.keras.Input(name='trip_start_month', shape=(1,), dtype=tf.string, sparse=False)
    k_hour  = tf.keras.Input(name='trip_start_hour', shape=(1,), dtype=tf.string, sparse=False)
    k_day  = tf.keras.Input(name='trip_start_day', shape=(1,), dtype=tf.string, sparse=False)
    k_picklat  = tf.keras.Input(name='pickup_latitude', shape=(1,), dtype=tf.float32, sparse=False)
    k_picklon  = tf.keras.Input(name='pickup_longitude', shape=(1,), dtype=tf.float32, sparse=False)
    k_droplat  = tf.keras.Input(name='dropoff_latitude', shape=(1,), dtype=tf.float32, sparse=False)
    k_droplon  = tf.keras.Input(name='dropoff_longitude', shape=(1,), dtype=tf.float32, sparse=False)
    k_distance  = tf.keras.Input(name='distance', shape=(1,), dtype=tf.float32, sparse=False)
    keras_dict_input = {'trip_start_month': k_month, 'trip_start_hour': k_hour, 'trip_start_day' : k_day,
                        'pickup_latitude': k_picklat, 'pickup_longitude': k_picklon,
                        'dropoff_latitude': k_droplat, 'dropoff_longitude': k_droplon, 'distance' : k_distance,
                        'pickup_longitude_scaled': k_pickup_longitude_scaled,
                        'pickup_latitude_scaled' : k_pickup_latitude_scaled
                        }

    return({'K' : keras_dict_input})

def create_keras_model(params, feature_cols):
    METRICS = [
            keras.metrics.RootMeanSquaredError(name='rmse')
    ]

    #Input layers
    input_feats = []
    for inp in feature_cols['K'].keys():
      input_feats.append(feature_cols['K'][inp])

    ##Input processing
    ##https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
    ##https://github.com/tensorflow/community/blob/master/rfcs/20191212-keras-categorical-inputs.md

    ##Handle categorical attributes( One-hot encoding )
    cat_day = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary = ['0'] + tf_transform_output.vocabulary_by_name(vocab_filename='trip_start_day'), mask_token=None, oov_token = '0')(feature_cols['K']['trip_start_day'])
    cat_day = tf.keras.layers.experimental.preprocessing.CategoryEncoding(num_tokens=tf_transform_output.vocabulary_size_by_name(vocab_filename='trip_start_day') + 1)(cat_day)

    cat_hour = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3','4','5','6','7','8'
                                                                                      '9','10','11','12','13','14','15','16',
                                                                                      '17','18','19','20','21','22','23','0'
                                                                                      ], mask_token=None)(feature_cols['K']['trip_start_hour'])
    cat_hour = tf.keras.layers.experimental.preprocessing.CategoryEncoding(num_tokens=24)(cat_hour)

    cat_month = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=['1','2','3','4','5','6','7','8'
                                                                                      '9','10','11','12'], mask_token=None)(feature_cols['K']['trip_start_month'])
    cat_month = tf.keras.layers.experimental.preprocessing.CategoryEncoding(num_tokens=12)(cat_month)

    # cat_company = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=df['company'].unique(), mask_token=None)(feature_cols['K']['company'])
    # cat_company = tf.keras.layers.experimental.preprocessing.CategoryEncoding(num_tokens=len(df['company'].unique()))(cat_company)

    ##Binning
    bins_pickup_lat = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lat)(feature_cols['K']['pickup_latitude'])
    cat_pickup_lat = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lat)+1)(bins_pickup_lat)

    bins_pickup_lon = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lon)(feature_cols['K']['pickup_longitude'])
    cat_pickup_lon = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lon)+1)(bins_pickup_lon)

    bins_drop_lat = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lat)(feature_cols['K']['dropoff_latitude'])
    cat_drop_lat = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lat)+1)(bins_drop_lat)

    bins_drop_lon = tf.keras.layers.experimental.preprocessing.Discretization(bins = bins_lon)(feature_cols['K']['dropoff_longitude'])
    cat_drop_lon = tf.keras.layers.experimental.preprocessing.CategoryEncoding(len(bins_lon)+1)(bins_drop_lon)

    ##Categorical cross
    cross_day_hour = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_day, cat_hour])
    hash_cross_day_hour = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=24 * 8)(cross_day_hour)
    cat_cross_day_hour = tf.keras.layers.experimental.preprocessing.CategoryEncoding(max_tokens = 24 * 8)(hash_cross_day_hour)

    cross_pick_lon_lat = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_pickup_lat, cat_pickup_lon])
    hash_cross_pick_lon_lat = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=(len(bins_lat) + 1) ** 2)(cross_pick_lon_lat)

    cross_drop_lon_lat = tf.keras.layers.experimental.preprocessing.CategoryCrossing()([cat_drop_lat, cat_drop_lon])
    hash_cross_drop_lon_lat = tf.keras.layers.experimental.preprocessing.Hashing(num_bins=(len(bins_lat) + 1) ** 2)(cross_drop_lon_lat)

    # Cross to embedding
    embed_cross_pick_lon_lat = tf.keras.layers.Embedding(((len(bins_lat) + 1) ** 2), 4)(hash_cross_pick_lon_lat)
    embed_cross_pick_lon_lat = tf.reduce_sum(embed_cross_pick_lon_lat, axis=-2)

    embed_cross_drop_lon_lat = tf.keras.layers.Embedding(((len(bins_lat) + 1) ** 2), 4)(hash_cross_drop_lon_lat)
    embed_cross_drop_lon_lat = tf.reduce_sum(embed_cross_drop_lon_lat, axis=-2)

    # Also pass time attributes as Deep signal( Cast to integer )
    int_trip_start_day = tf.strings.to_number(feature_cols['K']['trip_start_day'], tf.float32)
    int_trip_start_hour = tf.strings.to_number(feature_cols['K']['trip_start_hour'], tf.float32)
    int_trip_start_month = tf.strings.to_number(feature_cols['K']['trip_start_month'], tf.float32)

    #Add feature engineered columns - LAMBDA layer

    ###Create MODEL
    ####Concatenate all features( Numerical input )
    x_input_numeric = tf.keras.layers.concatenate([
                    feature_cols['K']['pickup_latitude'], feature_cols['K']['pickup_longitude'],
                    feature_cols['K']['dropoff_latitude'], feature_cols['K']['dropoff_longitude'],
                    feature_cols['K']['pickup_latitude_scaled'], feature_cols['K']['pickup_longitude_scaled'],
                    feature_cols['K']['distance'], embed_cross_pick_lon_lat, embed_cross_drop_lon_lat,
                    int_trip_start_day, int_trip_start_hour, int_trip_start_month
                    ])

    #DEEP - This Dense layer connects to input layer - Numeric Data
    x_numeric = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform")(x_input_numeric)
    x_numeric = tf.keras.layers.BatchNormalization()(x_numeric)

    ####Concatenate all Categorical features( Categorical converted )
    x_input_categ = tf.keras.layers.concatenate([
                    cat_month, cat_cross_day_hour, cat_pickup_lat, cat_pickup_lon,
                    cat_drop_lat, cat_drop_lon
                    ])
    
    #WIDE - This Dense layer connects to input layer - Categorical Data
    x_categ = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform")(x_input_categ)

    ####Concatenate both Wide and Deep layers
    x = tf.keras.layers.concatenate([x_categ, x_numeric])

    for l_ in range(params['hidden_layers']):
        x = tf.keras.layers.Dense(32, activation='relu', kernel_initializer="he_uniform",
                                  activity_regularizer=tf.keras.regularizers.l2(0.00001))(x)
        x = tf.keras.layers.BatchNormalization()(x)

    #Final Layer
    out = tf.keras.layers.Dense(1, activation='relu')(x)
    model = tf.keras.Model(input_feats, out)

    #Set optimizer
    opt = tf.keras.optimizers.Adam(lr= params['lr'], beta_1=params['beta_1'], 
                                        beta_2=params['beta_2'], epsilon=params['epsilon'])

    #Compile model
    model.compile(loss='mean_squared_error',  optimizer=opt, metrics = METRICS)

    #Print Summary
    print(model.summary())
    return model

def keras_train_and_evaluate(model, train_dataset, validation_dataset, epochs=100):
  #Add callbacks
  reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=5, min_lr=0.00001, verbose = 1)
  
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="/content/logs")

  #Train and Evaluate
  out = model.fit(train_dataset, 
                  validation_data = validation_dataset,
                  epochs=epochs,
                  # validation_steps = 3,   ###Keep this none for running evaluation on full EVAL data every epoch
                  steps_per_epoch = 100,   ###Has to be passed - Cant help it :) [ Number of batches per epoch ]
                  callbacks=[reduce_lr, #modelsave_callback, 
                             tensorboard_callback, 
                             keras.callbacks.EarlyStopping(patience=20, restore_best_weights=True, verbose=True)]
                  )
  
  return (min(out.history['val_rmse']))
  
@tf.function
def serving(dropoff_latitude, dropoff_longitude, pickup_latitude, pickup_longitude, trip_start_day, trip_start_hour, trip_start_month):
    #Params coming in request
    features = {
        'dropoff_latitude': dropoff_latitude,
        'dropoff_longitude': dropoff_longitude,
        'pickup_latitude': pickup_latitude,
        'pickup_longitude': pickup_longitude,
        'trip_start_day': trip_start_day,
        'trip_start_hour': trip_start_hour,
        'trip_start_month': trip_start_month
    }

    #Add TFT transformations
    raw_features = {}
    for key, val in features.items():
      if key not in RAW_DATA_FEATURE_SPEC:
        continue
      if isinstance(RAW_DATA_FEATURE_SPEC[key], tf.io.VarLenFeature):
        raw_features[key] = tf.RaggedTensor.from_tensor(
            tf.expand_dims(val, -1)).to_sparse()
        continue
      raw_features[key] = val
    tft_new_features = tft_layer(raw_features)

    pickup_longitude_scaled = tft_new_features['pickup_longitude_scaled'] 
    pickup_latitude_scaled = tft_new_features['pickup_latitude_scaled']
    distance = tft_new_features['distance']

    ##Feature engineering( calculate distance )
    # distance = tf.cast( tf.sqrt((tf.abs(dropoff_latitude - pickup_latitude))**2 + (tf.abs(dropoff_longitude - pickup_longitude))**2), tf.float32)

    #Params in request + New Feature engineering params
    payload = {
        'dropoff_latitude': dropoff_latitude,
        'dropoff_longitude': dropoff_longitude,
        'pickup_latitude': pickup_latitude,
        'pickup_longitude': pickup_longitude,
        'trip_start_day': trip_start_day,
        'trip_start_hour': trip_start_hour,
        'trip_start_month': trip_start_month,
        'distance': distance,
        'pickup_longitude_scaled': pickup_longitude_scaled,
        'pickup_latitude_scaled': pickup_latitude_scaled,
    }
    
    ## Predict
    ##IF THERE IS AN ERROR IN NUMBER OF PARAMS PASSED HERE OR DATA TYPE THEN IT GIVES ERROR, "COULDN'T COMPUTE OUTPUT TENSOR"
    predictions = m_(payload)
    return predictions

Writing task.py


In [46]:
# !rm -rf /content/transform_fn
# !rm -rf /content/*.csv

In [47]:
# !gsutil cp gs://rafiqhasandttl_01/taxi_dataset/train.csv /content/train.csv
# !gsutil cp gs://rafiqhasandttl_01/taxi_dataset/eval.csv /content/eval.csv

In [48]:
from google.colab import auth
auth.authenticate_user()

In [49]:
%%writefile -a task.py

##MAIN runner
print(tf.__version__)

device = "cpu"
if device == "tpu":
  resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
  tf.config.experimental_connect_to_cluster(resolver)
  # This is the TPU initialization code that has to be at the beginning.
  tf.tpu.experimental.initialize_tpu_system(resolver)
  strategy = tf.distribute.experimental.TPUStrategy(resolver)
else:
  strategy = tf.distribute.MultiWorkerMirroredStrategy()

#Constants
train_file = 'gs://rafiqhasandttl_01/taxi_dataset/train.csv' ##"/content/train.csv"
eval_file = 'gs://rafiqhasandttl_01/taxi_dataset/eval.csv' ##"/content/eval.csv"
transform_fn_output_dir= 'gs://rafiqhasandttl_01/ml_temporary/taxi_dataset/transform_fn'
model_save_location='gs://rafiqhasandttl_01/ml_models/taxi_dataset/'
epochs=3

# Determine CSV, label, and key columns
#Columns in training sheet -> Can have extra columns too
CSV_COLUMNS = ['fare', 'trip_start_month', 'trip_start_hour', 'trip_start_day',
       'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude']
LABEL_COLUMN = 'fare'

# Set default values for each CSV column( Including Y column )
DEFAULTS = [[0.0], ['1'], ['1'],['1'],[0.0],[0.0],[0.0],[0.0]]

bins_lat = [41.66367065, 41.85934972, 41.87740612, 41.87925508, 41.88099447,
       41.88498719, 41.88530002, 41.89204214, 41.89207263, 41.89265811,
       41.89830587, 41.89960211, 41.90026569, 41.90741282, 41.92187746,
       41.92926299, 41.9442266 , 41.95402765, 41.97907082, 42.02122359]

bins_lon = [-87.9136246 , -87.76550161, -87.68751552, -87.67161972,
       -87.66341641, -87.65599818, -87.65253448, -87.64629348,
       -87.642649  , -87.63784421, -87.63330804, -87.63274649,
       -87.63186395, -87.62887416, -87.62621491, -87.62519214,
       -87.62099291, -87.62076287, -87.61886836, -87.54093551]

RAW_DATA_FEATURE_SPEC = dict([
        ('fare', tf.io.VarLenFeature(tf.float32)),
        ('trip_start_month', tf.io.VarLenFeature(tf.string)),
        ('trip_start_hour', tf.io.VarLenFeature(tf.string)),
        ('trip_start_day', tf.io.VarLenFeature(tf.string)),
        ('pickup_latitude', tf.io.FixedLenFeature([], tf.float32)),
        ('pickup_longitude', tf.io.FixedLenFeature([], tf.float32)),
        ('dropoff_latitude', tf.io.FixedLenFeature([], tf.float32)),
        ('dropoff_longitude', tf.io.FixedLenFeature([], tf.float32)),
        ])

#Run TFT Transformation
beam_run_transform_data(train_data_file=train_file, transform_fn_output_dir=transform_fn_output_dir)

#Create TFT layer
tf_transform_output = tft.TFTransformOutput(transform_fn_output_dir)
tft_layer = tf_transform_output.transform_features_layer()

#Create FC
create_feature_cols()

#Create model
params_default = {
    'lr' : 0.001,
    'beta_1' : 0.99,
    'beta_2' : 0.999,
    'epsilon' : 1e-08,
    'decay' : 0.01,
    'hidden_layers' : 1
}

#Create dataset input functions
train_dataset = make_input_fn(filename = train_file,
                    mode = tf.estimator.ModeKeys.TRAIN,
                    batch_size = 128)()

validation_dataset = make_input_fn(filename = eval_file,
                    mode = tf.estimator.ModeKeys.EVAL,
                    batch_size = 512)()

m_ = create_keras_model(params = params_default, feature_cols = create_feature_cols())
tf.keras.utils.plot_model(m_, show_shapes=True, rankdir="LR")

#Train Model
rmse = keras_train_and_evaluate(m_, train_dataset, validation_dataset, epochs)
print("Final Val RMSE: ", rmse)

#Save model
serving = serving.get_concrete_function(trip_start_day=tf.TensorSpec([None,], dtype= tf.string, name='trip_start_day'), 
                                        trip_start_hour=tf.TensorSpec([None,], dtype= tf.string, name='trip_start_hour'),
                                        trip_start_month=tf.TensorSpec([None], dtype= tf.string, name='trip_start_month'), 
                                        dropoff_latitude=tf.TensorSpec([None,], dtype= tf.float32, name='dropoff_latitude'),
                                        dropoff_longitude=tf.TensorSpec([None,], dtype= tf.float32, name='dropoff_longitude'), 
                                        pickup_latitude=tf.TensorSpec([None,], dtype= tf.float32, name='pickup_latitude'),
                                        pickup_longitude=tf.TensorSpec([None,], dtype= tf.float32, name='pickup_longitude')
                                        )

version = "1"  #{'serving_default': call_output}
tf.saved_model.save(
    m_,
    model_save_location + version,
    signatures=serving
) 

Appending to task.py


In [50]:
!python task.py

2021-08-24 16:13:52.825224: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2.5.1
2021-08-24 16:13:54.348377: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-24 16:13:54.361983: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-08-24 16:13:54.362038: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (0af8948a189e): /proc/driver/nvidia/version does not exist
2021-08-24 16:13:54.362526: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### **Vertex Tensorboard logging**

In [71]:
%%bash 
gcloud config set project us-gcp-ame-con-06f-npd-1

# tb-gcp-uploader --help

tb-gcp-uploader --tensorboard_resource_name "projects/382989511509/locations/us-central1/tensorboards/4093385033186803712" \
  --logdir=gs://rafiqhasandttl_01/ml_tensorlogs/ \
  --experiment_name=h1 --one_shot=True

INFO:tensorboard:Creating experiment
View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+382989511509+locations+us-central1+tensorboards+4093385033186803712+experiments+h1
INFO:tensorboard:Starting an upload cycle
INFO:tensorboard:Starting logdir traversal of gs://rafiqhasandttl_01/ml_tensorlogs/
INFO:tensorboard:GetLogdirSubdirectories: Starting to list directories via glob-ing.
INFO:tensorboard:GlobAndListFiles: Starting to glob level 0
INFO:tensorboard:GlobAndListFiles: 2 files glob-ed at level 0
INFO:tensorboard:GlobAndListFiles: Starting to glob level 1
INFO:tensorboard:GlobAndListFiles: 4 files glob-ed at level 1
INFO:tensorboard:- Adding run for relative directory train
INFO:tensorboard:- Adding run for relative directory validation
INFO:tensorboard:GlobAndListFiles: Starting to glob level 2
INFO:tensorboard:GlobAndListFiles: 1 files glob-ed at level 2
INFO:tensorboard:GlobAndListFiles: Starting to glob level 3
INFO:tensorboard:Glob

Updated property [core/project].
2021-08-24 16:45:45.260240: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
W0824 16:45:46.777312 139665146435456 _default.py:484] No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
I0824 16:45:46.777934 139665146435456 uploader.py:262] Creating experiment
I0824 16:45:46.943666 139665146435456 uploader.py:326] Starting an upload cycle
I0824 16:45:46.944006 139665146435456 logdir_loader.py:65] Starting logdir traversal of gs://rafiqhasandttl_01/ml_tensorlogs/
I0824 16:45:48.141810 139665146435456 io_wrapper.py:216] GetLogdirSubdirectories: Starting to list directories via glob-ing.
I0824 16:45:48.142232 139665146435456 io_wrapper.py:133] GlobAndListFiles: Starting to glob level 0
I0824 16:45:48.209699 139665146435456 io_wrapper.py:136] GlobAndListFiles: 2 files glob-ed at level 0
I0824 16:45:48.210104

### **Packaging Trainer Script**

In [8]:
%%writefile setup.py

from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = ['absl-py','folium', 'pandas', 'tensorflow_transform',
                     'google-cloud','google-cloud-storage','google-cloud-firestore','google-api-python-client', 'google-auth']

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Hasan - Vertex AI Taxi Trainer Job'
)

Writing setup.py


In [9]:
%%writefile __init__.py
 

Writing __init__.py


In [10]:
##Create trainer package
!rm -rf /content/trainer
!rm -rf /content/dist/
!rm -rf /content/trainer*

!mkdir /content/trainer/
!cp /content/task.py /content/trainer/
!cp /content/__init__.py /content/trainer/
!python /content/setup.py sdist

running sdist
running egg_info
creating trainer.egg-info
writing trainer.egg-info/PKG-INFO
writing dependency_links to trainer.egg-info/dependency_links.txt
writing requirements to trainer.egg-info/requires.txt
writing top-level names to trainer.egg-info/top_level.txt
writing manifest file 'trainer.egg-info/SOURCES.txt'
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'

running check


creating trainer-0.1
creating trainer-0.1/trainer
creating trainer-0.1/trainer.egg-info
copying files to trainer-0.1...
copying setup.py -> trainer-0.1
copying trainer/__init__.py -> trainer-0.1/trainer
copying trainer/task.py -> trainer-0.1/trainer
copying trainer.egg-info/PKG-INFO -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/SOURCES.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/dependency_links.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-info/requires.txt -> trainer-0.1/trainer.egg-info
copying trainer.egg-in

In [11]:
# Copy trainer.gz to GCS training path
!gsutil cp /content/dist/trainer-0.1.tar.gz gs://rafiqhasandttl_01/ml_scripts/taxi_dataset/trainer/

Copying file:///content/dist/trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  7.2 KiB/  7.2 KiB]                                                
Operation completed over 1 objects/7.2 KiB.                                      


### **Creating KFP Vertex Pipeline**

In [24]:
import json
import logging
from google.colab import auth
from typing import NamedTuple

import kfp
# from google.cloud import aiplatform
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import dsl
from kfp.v2 import compiler  # noqa: F811
from kfp.v2.dsl import (ClassificationMetrics, Input, Metrics, Model, Output, Dataset,
                        component)
from kfp.v2.google.client import AIPlatformClient

auth.authenticate_user()

PROJECT_ID = 'us-gcp-ame-con-06f-npd-1'
DISPLAY_NAME = 'taxi-custom-vertex-pipeline'
REGION = 'us-central1'
PIPELINE_ROOT = 'gs://rafiqhasandttl_01/ml_pipelines/'

In [30]:
@component(
    packages_to_install = [
        "tensorflow_transform","tensorflow==2.5.0"
    ],
)
def run_tft_transform(
    train_data_file: str,
    transform_fn_output_dir: str,
    metrics: Output[Metrics],
    mlmd_dataset: Output[Dataset]
) -> NamedTuple("Outputs", [("fn_output_dir", str)]): ##Output of COMPONENT
    
    import apache_beam as beam
    import tensorflow_transform as tft
    import tempfile
    import tensorflow_transform.beam as tft_beam

    from tfx_bsl.public import tfxio
    from tensorflow_transform.tf_metadata import dataset_metadata
    from tensorflow_transform.tf_metadata import schema_utils

    import tensorflow as tf
    import tensorflow.keras as keras

    CSV_COLUMNS = ['fare', 'trip_start_month', 'trip_start_hour', 'trip_start_day',
       'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
       'dropoff_longitude']

    RAW_DATA_FEATURE_SPEC = dict([
        ('fare', tf.io.VarLenFeature(tf.float32)),
        ('trip_start_month', tf.io.VarLenFeature(tf.string)),
        ('trip_start_hour', tf.io.VarLenFeature(tf.string)),
        ('trip_start_day', tf.io.VarLenFeature(tf.string)),
        ('pickup_latitude', tf.io.FixedLenFeature([], tf.float32)),
        ('pickup_longitude', tf.io.FixedLenFeature([], tf.float32)),
        ('dropoff_latitude', tf.io.FixedLenFeature([], tf.float32)),
        ('dropoff_longitude', tf.io.FixedLenFeature([], tf.float32)),
        ])

    # https://www.tensorflow.org/tfx/tutorials/transform/census
    def tf_preprocessing_fn(inputs):
        """Preprocess input columns into transformed columns."""
        pickup_latitude = inputs['pickup_latitude']
        pickup_longitude = inputs['pickup_longitude']

        #Calculate unique values of attribute
        tft.vocabulary(inputs['trip_start_day'], vocab_filename='trip_start_day')

        pickup_latitude_scaled = tft.scale_to_0_1(pickup_latitude)
        pickup_longitude_scaled = tft.scale_to_0_1(pickup_longitude)
        distance = ((inputs['pickup_latitude'] - inputs['dropoff_latitude'])**2 +  (inputs['pickup_longitude'] - inputs['dropoff_longitude'])**2)**0.5

        return {
            'pickup_latitude_scaled': pickup_latitude_scaled,
            'pickup_longitude_scaled': pickup_longitude_scaled,
            'distance': distance,
            # 'dummy1': dummy1
        }

    def beam_run_transform_data(train_data_file, transform_fn_output_dir):
      """Transform the data and write out as a TFRecord of Example protos.

      Read in the data using the CSV reader, and transform it using a
      preprocessing pipeline that scales numeric data and converts categorical data
      from strings to int64 values indices, by creating a vocabulary for each
      category.

      Args:
        train_data_file: File containing training data
        transform_fn_output_dir: Directory to write function
      """
    
      SCHEMA = tft.tf_metadata.dataset_metadata.DatasetMetadata(
        tft.tf_metadata.schema_utils.schema_from_feature_spec(RAW_DATA_FEATURE_SPEC)).schema

      # The "with" block will create a pipeline, and run that pipeline at the exit
      # of the block.
      with beam.Pipeline() as pipeline:
        with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
          # Create a TFXIO to read the census data with the schema. To do this we
          # need to list all columns in order since the schema doesn't specify the
          # order of columns in the csv.
          # We first read CSV files and use BeamRecordCsvTFXIO whose .BeamSource()
          # accepts a PCollection[bytes] because we need to patch the records first
          # (see "FixCommasTrainData" below). Otherwise, tfxio.CsvTFXIO can be used
          # to both read the CSV files and parse them to TFT inputs:
          # csv_tfxio = tfxio.CsvTFXIO(...)
          # raw_data = (pipeline | 'ToRecordBatches' >> csv_tfxio.BeamSource())
          csv_tfxio = tfxio.BeamRecordCsvTFXIO(
              physical_format='text',
              column_names=CSV_COLUMNS,
              schema=SCHEMA)

          # Read in raw data and convert using CSV TFXIO.  Note that we apply
          # some Beam transformations here, which will not be encoded in the TF
          # graph since we don't do the from within tf.Transform's methods
          # (AnalyzeDataset, TransformDataset etc.).  These transformations are just
          # to get data into a format that the CSV TFXIO can read, in particular
          # removing spaces after commas.
          raw_data = (
              pipeline
              | 'ReadTrainData' >> beam.io.ReadFromText(
                  train_data_file, coder=beam.coders.BytesCoder())
              | 'FixCommasTrainData' >> beam.Map(
                  lambda line: line.replace(b', ', b','))
              | 'DecodeTrainData' >> csv_tfxio.BeamSource())

          # Combine data and schema into a dataset tuple.  Note that we already used
          # the schema to read the CSV data, but we also need it to interpret
          # raw_data.
          raw_dataset = (raw_data, csv_tfxio.TensorAdapterConfig())

          # The TFXIO output format is chosen for improved performance.
          transform_fn = (
              raw_dataset | tft_beam.AnalyzeDataset(tf_preprocessing_fn))

          # Will write a SavedModel and metadata to transform_fn_output_dir, which can then
          # be read by the tft.TFTransformOutput class.
          _ = (
              transform_fn
              | 'WriteTransformFn' >> tft_beam.WriteTransformFn(transform_fn_output_dir))
          
    
    #Main runner
    beam_run_transform_data(train_data_file, transform_fn_output_dir)
    metrics.log_metric("size", "dummy-1")                               #Saving metric
    mlmd_dataset.metadata["metadata"] = "value-1"                        #Saving Metadata

    return (transform_fn_output_dir, )

In [35]:
@kfp.dsl.pipeline(name="hasan-custom-vertex-pipeline-taxi", pipeline_root=PIPELINE_ROOT)
def pipeline(
    display_name: str = DISPLAY_NAME,
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
    train_file: str = 'gs://rafiqhasandttl_01/taxi_dataset/train.csv', ##"/content/train.csv"
    eval_file: str = 'gs://rafiqhasandttl_01/taxi_dataset/eval.csv', ##"/content/eval.csv"
    transform_fn_output_dir: str = 'gs://rafiqhasandttl_01/ml_temporary/taxi_dataset/transform_fn',
    model_save_location: str = 'gs://rafiqhasandttl_01/ml_models/taxi_dataset/',
    epochs: int = 1
):

    import uuid
    import json

    # Determine CSV, label, and key columns
    #Columns in training sheet -> Can have extra columns too
    # CSV_COLUMNS = ['fare', 'trip_start_month', 'trip_start_hour', 'trip_start_day',
    #       'pickup_latitude', 'pickup_longitude', 'dropoff_latitude',
    #       'dropoff_longitude']
    # LABEL_COLUMN = 'fare'

    # # Set default values for each CSV column( Including Y column )
    # DEFAULTS = [[0.0], ['1'], ['1'],['1'],[0.0],[0.0],[0.0],[0.0]]

    # bins_lat = [41.66367065, 41.85934972, 41.87740612, 41.87925508, 41.88099447,
    #       41.88498719, 41.88530002, 41.89204214, 41.89207263, 41.89265811,
    #       41.89830587, 41.89960211, 41.90026569, 41.90741282, 41.92187746,
    #       41.92926299, 41.9442266 , 41.95402765, 41.97907082, 42.02122359]

    # bins_lon = [-87.9136246 , -87.76550161, -87.68751552, -87.67161972,
    #       -87.66341641, -87.65599818, -87.65253448, -87.64629348,
    #       -87.642649  , -87.63784421, -87.63330804, -87.63274649,
    #       -87.63186395, -87.62887416, -87.62621491, -87.62519214,
    #       -87.62099291, -87.62076287, -87.61886836, -87.54093551]

    # RAW_DATA_FEATURE_SPEC = dict([
    #         ('fare', tf.io.VarLenFeature(tf.float32)),
    #         ('trip_start_month', tf.io.VarLenFeature(tf.string)),
    #         ('trip_start_hour', tf.io.VarLenFeature(tf.string)),
    #         ('trip_start_day', tf.io.VarLenFeature(tf.string)),
    #         ('pickup_latitude', tf.io.FixedLenFeature([], tf.float32)),
    #         ('pickup_longitude', tf.io.FixedLenFeature([], tf.float32)),
    #         ('dropoff_latitude', tf.io.FixedLenFeature([], tf.float32)),
    #         ('dropoff_longitude', tf.io.FixedLenFeature([], tf.float32)),
    #         ])

    #STEP 1: Analyze Data using BEAM and create transformation fn
    tft_op = run_tft_transform(
                  train_data_file = train_file,
                  transform_fn_output_dir= transform_fn_output_dir,
              )

    #STEP 2: For non Auto-ML call
    training_args = ['--tranform_fn_dir', str(tft_op.outputs['fn_output_dir'])]

    job_name = "hasan-vertex-custom-" + str(uuid.uuid4())
    training_op = gcc_aip.CustomPythonPackageTrainingJobRunOp(
            project=PROJECT_ID,
            display_name=job_name,
            # model_display_name="hasan-vertex-taxi",
            python_package_gcs_uri="gs://rafiqhasandttl_01/ml_scripts/taxi_dataset/trainer/trainer-0.1.tar.gz",
            python_module="trainer.task",
            container_uri='us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-5:latest',
            staging_bucket='gs://rafiqhasandttl_01/ml_temporary/',
            # base_output_dir=output_dir,
            # model_serving_container_image_uri=model_serving_container_image_uri,
            replica_count=1,
            location="us-east4",
            machine_type="n1-standard-4",
            args=training_args)

In [36]:
!rm -rf /content/*pipeline*.json

In [37]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="/content/hasan-custom-vertex-pipeline-taxi.json"
)

In [38]:
from kfp.v2.google.client import AIPlatformClient

api_client = AIPlatformClient(
                project_id=PROJECT_ID,
                region=REGION
                )

response = api_client.create_run_from_job_spec(
    '/content/hasan-custom-vertex-pipeline-taxi.json',
)

/usr/local/lib/python3.7/dist-packages/kfp/v2/google/client/client.py:175: FutureWarning: AIPlatformClient will be deprecated in v1.9. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


### **Local TMS Prediction**

In [85]:
###Install TF Model server

# This is the same as you would do from your command line, but without the [arch=amd64], and no sudo
# You would instead do:
# echo "deb [arch=amd64] http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | sudo tee /etc/apt/sources.list.d/tensorflow-serving.list && \
# curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | sudo apt-key add -

!apt-get remove tensorflow-model-server
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

!apt-get install tensorflow-model-server

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package tensorflow-model-server
deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2943  100  2943    0     0  57705      0 --:--:-- --:--:-- --:--:-- 57705
OK
Get:1 http://storage.googleapis.com/tensorflow-serving-apt stable InRelease [3,012 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-lear

In [94]:
import json
import requests

#Create payload
data_py = {"instances":[{'dropoff_latitude': 41.920452,
                         'dropoff_longitude': -87.679955,
                         'pickup_latitude': 41.952823,
                         'pickup_longitude': -87.653244,
                         'trip_start_day': "1",
                         'trip_start_hour': "5",
                         'trip_start_month': "6"}]}

#Works as well
# data_py = {"inputs":{'dropoff_latitude': [41.920452],
#                          'dropoff_longitude': [-87.679955],
#                          'pickup_latitude': [41.952823],
#                          'pickup_longitude': [-87.653244],
#                          'trip_start_day': ["1"],
#                          'trip_start_hour': ["5"],
#                          'trip_start_month': ["6"]}}

data = json.dumps(data_py)
print("payload: ", data)

#Run request on TMS
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8505/v1/models/model:predict', data=data, headers=headers)
json_response.text

payload:  {"instances": [{"dropoff_latitude": 41.920452, "dropoff_longitude": -87.679955, "pickup_latitude": 41.952823, "pickup_longitude": -87.653244, "trip_start_day": "1", "trip_start_hour": "5", "trip_start_month": "6"}]}


'{\n    "predictions": [[97.8500519]\n    ]\n}'

In [86]:
###Start Tensorflow server
# %%bash --bg 
# export TF_CPP_MIN_VLOG_LEVEL=0

%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8505 \
  --model_name=model \
  --model_base_path="gs://rafiqhasandttl_01/ml_models/taxi_dataset/" >server.log 2>&1

Starting job # 0 in a separate thread.


In [87]:
!tail server.log

2021-08-24 17:19:54.426475: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:283] SavedModel load for tags { serve }; Status: success: OK. Took 1959423 microseconds.
2021-08-24 17:19:54.553369: I tensorflow_serving/servables/tensorflow/saved_model_warmup_util.cc:59] No warmup data file found at gs://rafiqhasandttl_01/ml_models/taxi_dataset/1/assets.extra/tf_serving_warmup_requests
2021-08-24 17:19:55.193983: I tensorflow_serving/core/loader_harness.cc:87] Successfully loaded servable version {name: model version: 1}
2021-08-24 17:19:55.196968: I tensorflow_serving/model_servers/server_core.cc:486] Finished adding/updating models
2021-08-24 17:19:55.197053: I tensorflow_serving/model_servers/server.cc:133] Using InsecureServerCredentials
2021-08-24 17:19:55.197069: I tensorflow_serving/model_servers/server.cc:383] Profiler service is enabled
2021-08-24 17:19:55.197652: I tensorflow_serving/model_servers/server.cc:409] Running gRPC ModelServer at 0.0.0.0:8500 ...
[warn] geta

### **Vertex AIP model prediction**

In [83]:
%%writefile input.json

{
   "instances":[
      {
         "dropoff_latitude":41.920452,
         "dropoff_longitude":-87.679955,
         "pickup_latitude":41.952823,
         "pickup_longitude":-87.653244,
         "trip_start_day":"1",
         "trip_start_hour":"5",
         "trip_start_month":"6"
      }
   ]
}

Overwriting input.json


In [84]:
%%bash
ENDPOINT_ID="6416785044072824832"
PROJECT_ID="us-gcp-ame-con-06f-npd-1"
INPUT_DATA_FILE="input.json"

curl \
-X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
https://us-east4-aiplatform.googleapis.com/v1/projects/${PROJECT_ID}/locations/us-east4/endpoints/${ENDPOINT_ID}:predict \
-d "@${INPUT_DATA_FILE}"

{
  "predictions": [
    [
      97.8500519
    ]
  ],
  "deployedModelId": "7539940569892519936"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   382    0   100  100   282    305    862 --:--:-- --:--:-- --:--:--  1168
